# make_fgmax_grid_Newport

For [Cascadia CoPes Hub](https://cascadiacopeshub.org/) tsunami modeling, sponsored by NSF.

Creates an fgmax grid for point_style==4 in GeoClaw covering only points around Yaquina Bay that are below some specified elevation.

For a notebook with more explanation, see [MarchingFront.ipynb](https://www.clawpack.org/gallery/_static/apps/notebooks/geoclaw/MarchingFront.html) in the Clawpack Gallery of Jupyter notebooks, and 

R.J. LeVeque 1/30/25

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os,sys
from clawpack.visclaw import colormaps, plottools
from clawpack.geoclaw import topotools, marching_front, kmltools, fgmax_tools
from clawpack.amrclaw import region_tools
from numpy import ma
import glob
import zipfile

In [ ]:
zmin = -70.
zmax = 35.
sea_level = 0.  # corresponding to MHW in coastal DEMs

cmap_land = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})

cmap_water = colormaps.make_colormap({ 0.0:[0,0,1], 1.:[.8,.8,1]})

cmap_topo, norm_topo = colormaps.add_colormaps((cmap_land, cmap_water),
                                     data_limits=(zmin,zmax),
                                     data_break=sea_level)

cmap_below_sea_level_dry = colormaps.make_colormap({ 0.0:[1.0,0.7,0.7], 1.:[1.0,0.7,0.7]})
cmap_dry, norm_dry = colormaps.add_colormaps((cmap_land, cmap_below_sea_level_dry),
                                     data_limits=(zmin,zmax),
                                     data_break=sea_level) 

def plottopo(topo):
    figure(figsize=(13,8))
    plottools.pcolorcells(topo.X, topo.Y, topo.Z, cmap=cmap_topo, norm=norm_topo)
    colorbar(extend='both',shrink=0.5)
    gca().set_aspect(1./cos(44.6*pi/180.))
    xticks(rotation=20);

In [ ]:
fname = 'topofiles/Newport_13s_mhw.asc'
topo = topotools.Topography()
topo.read(fname, topo_type=3)
print('Extent: \n', topo.extent)
print('Offset in x = %.3f cells' % ((topo.x[0]+124)*3*3600))
print('Offset in y = %.3f cells' % ((topo.y[0]-44)*3*3600))
print('These should be integers if aligned so DEM point is at integer lat-long')

In [ ]:
loc = 'Newport'
fgmax_extent = [-124.1, -124, 44.565, 44.642]
topo_fgmax = topo.crop(fgmax_extent)
topo_fgmax.X.shape

In [ ]:
# not used here:
RR_extent = None
mask_outside_extent = None

## Select fgmax points below some elevation

In [ ]:
fgmax_pts_chosen = marching_front.select_by_flooding(topo_fgmax.Z, 
                               Z1=0, Z2=1e6, max_iters=20,mask=mask_outside_extent) 

In [ ]:
fgmax_pts_chosen = marching_front.select_by_flooding(topo_fgmax.Z, 
                                   Z1=0, Z2=35., prev_pts_chosen=fgmax_pts_chosen,
                                   max_iters=None,mask=mask_outside_extent)

In [ ]:
Z_fgmax = ma.masked_array(topo_fgmax.Z,  logical_not(fgmax_pts_chosen))

In [ ]:
topo_fgmax.Z.shape, fgmax_pts_chosen.shape

In [ ]:
fgmax_pts_chosen = where(fgmax_pts_chosen, 1, 0)  # change boolean to 1/0
topo_fgmax_pts = topotools.Topography()
topo_fgmax_pts.set_xyZ(topo_fgmax.x, topo_fgmax.y, fgmax_pts_chosen)

In [ ]:
figure(figsize=(13,8))
plottools.pcolorcells(topo_fgmax.X, topo_fgmax.Y, Z_fgmax, cmap=cmap_topo, norm=norm_topo)
colorbar(extend='both',shrink=0.5)
gca().set_aspect(1./cos(48*pi/180.))
xticks(rotation=20);
ticklabel_format(style='plain',useOffset=False);

In [ ]:
fname_fgmax_pts = 'fgmax_pts_%s.data' % loc
topo_fgmax_pts.write(fname_fgmax_pts, topo_type=3, Z_format='%1i')
print('Created %s' % fname_fgmax_pts)

### Make kml figures and files

In [ ]:
kml_dir = 'fgmax_kmlfiles_%s' % loc
os.system('mkdir -p %s' % kml_dir)
print('Will put png and kml files in %s' % kml_dir)

In [ ]:
# Make kml files showing extent of topo_fgmax:
kml_extent_dir = os.path.join(kml_dir, 'extents')
os.system('mkdir -p %s' % kml_extent_dir)
print('Will put extent kml files in %s' % kml_extent_dir)

name = loc + ' extent of topo_fgmax'
fname_topo_fgmax = '%s/topo_fgmax_extent.kml' % kml_extent_dir
kmltools.box2kml(topo_fgmax.extent, fname_topo_fgmax, name,
                 color='00FF00',width=1,verbose=True)

RR_extent = None
if RR_extent:
    # if a ruled rectangle further restricts fgmax points, make it's kml:
    name = loc + ' RR_extent for fgmax'
    fname_RR_fgmax = '%s/RR_extent_fgmax.kml' % kml_extent_dir
    RR_extent.make_kml(fname_RR_fgmax, name, color='00FFFF', width=2, verbose=True)
    

In [ ]:
Z_land = ma.masked_where(Z_fgmax<0., Z_fgmax)
png_filename = '%s/fgmax_%s_land.png' % (kml_dir, loc)
fig,ax,png_extent,kml_dpi = kmltools.pcolorcells_for_kml(topo_fgmax.X, topo_fgmax.Y, Z_land,
                                                 png_filename=png_filename,
                                                 dpc=2, cmap=cmap_topo, norm=norm_topo)

In [ ]:
# Create a version of topo_fgmax.Z with all wet points masked out:

wet_points = marching_front.select_by_flooding(Z_fgmax, Z1=-5., Z2=0., max_iters=None,
                                               mask=mask_outside_extent)
mask_dry = logical_not(wet_points)
Z_dry = ma.masked_array(Z_fgmax, wet_points)

In [ ]:
Z_water = ma.masked_where(mask_dry, Z_fgmax)
png_filename = '%s/fgmax_%s_water.png' % (kml_dir, loc)
fig,ax,png_extent,kml_dpi = kmltools.pcolorcells_for_kml(topo_fgmax.X, topo_fgmax.Y, Z_water,
                                                 png_filename=png_filename,
                                                 dpc=2, cmap=cmap_topo, norm=norm_topo)

In [ ]:
kmltools.kml_build_colorbar('%s/fgmax_colorbar.png' % kml_dir, cmap_topo, 
                           norm=norm_topo, label='meters', title='topo', extend='min')

In [ ]:
png_files=['fgmax_%s_water.png' % loc, 
           'fgmax_%s_land.png' % loc]
png_names=['fgmax_%s_water' % loc,
           'fgmax_%s_land' % loc]
cb_files = ['fgmax_colorbar.png']
cb_names = ['colorbar_topo']

name = 'fgmax_%s_topo' % loc
fname = os.path.join(kml_dir, name+'.kml')
kmltools.png2kml(png_extent, png_files=png_files, png_names=png_names, 
                 name=name, fname=fname,
                 radio_style=False,
                 cb_files=cb_files, cb_names=cb_names)

In [ ]:
savedir = os.getcwd()
os.chdir(kml_dir)
files = glob.glob('*.kml') + glob.glob('*.png')
print('kmz file will include:')
for file in files:
    print('    %s' % os.path.split(file)[-1])

fname_kmz = 'fgmax_topo_%s.kmz' % loc
with zipfile.ZipFile(fname_kmz, 'w') as zip:
    for file in files:
        zip.write(file) 
    print('Created %s' % os.path.abspath(fname_kmz))
os.chdir(savedir)